---
toc: true
comments: true
layout: post
title: Stocks Project
courses: { csa: {week: 2} }
type: hacks
permalink: /stocks
---

<html>
<body>
  <div>
    <!-- Input for the stock symbol -->
    <label for="stockInput">Enter Stock Symbol: </label>
    <input type="text" id="stockInput" placeholder="e.g. MSFT">
  <div class="dropdown">
  <button class="dropbtn">Month</button>
  <div class="dropdown-content">
    <a href="#">January</a>
    <a href="#">February</a>
    <a href="#">March</a>
    <a href="#">April</a>
    <a href="#">May</a>
    <a href="#">June</a>
    <a href="#">July</a>
    <a href="#">August</a>
    <a href="#">September</a>
    <a href="#">October</a>
    <a href="#">November</a>
    <a href="#">December</a>
</div>

<style>
/* Style The Dropdown Button */
.dropbtn {
  background-color: #4CAF50;
  color: white;
  padding: 16px;
  font-size: 16px;
  border: none;
  cursor: pointer;
}

/* The container <div> - needed to position the dropdown content */
.dropdown {
  position: relative;
  display: inline-block;
}

/* Dropdown Content (Hidden by Default) */
.dropdown-content {
  display: none;
  position: absolute;
  background-color: #f9f9f9;
  min-width: 160px;
  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);
  z-index: 1;
}

/* Links inside the dropdown */
.dropdown-content a {
  color: black;
  padding: 12px 16px;
  text-decoration: none;
  display: block;
}

/* Change color of dropdown links on hover */
.dropdown-content a:hover {background-color: #f1f1f1}

/* Show the dropdown menu on hover */
.dropdown:hover .dropdown-content {
  display: block;
}

/* Change the background color of the dropdown button when the dropdown content is shown */
.dropdown:hover .dropbtn {
  background-color: #3e8e41;
}
</style>

  <!-- Canvas element for the graph -->
  <canvas></canvas>

  <div class="prediction-wrapper" style="display:none">
    <!-- Input for user to enter a date for prediction -->
    <label for="predictionInput">Enter a Month to Predict your Selected Stock Price (YYYY-MM): </label>
    <input type="text" id="predictionInput" placeholder="e.g. 2023-09">
    <button onclick="predictPrice()">Predict Price</button>
    <!-- Display the prediction result -->
    <h1 id="prediction"></h1>
  </div>

  <script>

    // Global variables to store regression data and x-value
    let regression = {}
    let x = 0
    let ctx;
    // Function to fetch stock data
    async function fetchData() {
      const stockSymbol = document.getElementById('stockInput').value;
      const url = `https://alpha-vantage.p.rapidapi.com/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=${stockSymbol}&datatype=json`;
      const options = {
        method: 'GET',
        headers: {
          'X-RapidAPI-Key': '115318933dmsh6d4dd26b9c05b57p138eb6jsn5766c7fe61da',
          'X-RapidAPI-Host': 'alpha-vantage.p.rapidapi.com'
        }
      };

      try {
        // Fetch stock data from the API
        const response = await fetch(url, options);
        const result = await response.json();

        // Extract dates and adjusted closing prices from the API response
        const weeklyData = result['Monthly Adjusted Time Series'];
        const dates = Object.keys(weeklyData);

        // Parse prices as floats
        const prices = dates.map((date) => parseFloat(weeklyData[date]['5. adjusted close']));

        // Get canvas element and context
        const canvas = document.getElementsByTagName('canvas')[0];
        ctx = canvas.getContext('2d');

        // Generate the graph using the data
        generateGraph(dates, prices)

      } catch (error) {
        console.error(error);
      }
    }

    // Function to generate the graph
    function generateGraph(dates, prices) {
      // Get canvas element and context
      const canvas = document.getElementsByTagName('canvas')[0];

      // Define canvas dimensions
      canvas.width = 700;
      canvas.height = 400;

      // Define data points and chart dimensions
      const dataPoints = prices.length;
      const margin = 40;
      const chartWidth = canvas.width - 2 * margin;
      const chartHeight = canvas.height - 2 * margin;

      // Calculate scaling factors
      const maxPrice = Math.max(...prices);
      const minPrice = Math.min(...prices);
      const priceRange = maxPrice - minPrice;
      const xStep = chartWidth / (dataPoints - 1);
      const yStep = chartHeight / priceRange;

      // Draw x and y axes
      ctx.beginPath();
      ctx.moveTo(margin, margin);
      ctx.lineTo(margin, canvas.height - margin);
      ctx.lineTo(canvas.width - margin, canvas.height - margin);
      ctx.stroke();

      // Plot stock prices
      ctx.beginPath();
      ctx.strokeStyle = 'blue';
      ctx.lineWidth = 2;
      for (let i = dataPoints; i > 0; i--) {
        const x = canvas.width - 40 - i * xStep;
        const y = canvas.height - margin - (prices[i] - minPrice) * yStep;
        if (i === 0) {
          ctx.moveTo(x, y);
        } else {
          ctx.lineTo(x, y);
        }
      }
      ctx.stroke();

      // Calculate linear regression
      const xValues = [...Array(dataPoints).keys()];
      const yValues = prices;
      regression = linearRegression(xValues, yValues);

      // Plot linear regression line
      ctx.beginPath();
      ctx.strokeStyle = 'red';
      ctx.lineWidth = 2;
      for (let i = 0; i < dataPoints; i++) {
        const x = canvas.width - 40 - i * xStep;
        const y = canvas.height - margin - (regression.slope * i + regression.intercept - minPrice) * yStep;
        if (i === 0) {
          ctx.moveTo(x, y);
        } else {
          ctx.lineTo(x, y);
        }
      }
      ctx.stroke();

      // Label axes
      ctx.fillStyle = 'black';
      ctx.fillText('Date', canvas.width - margin - 20, canvas.height - 10);
      ctx.fillText('Stock Price', margin - 10, margin - 10);

      // Display the prediction input section
      document.getElementsByClassName("prediction-wrapper")[0].style.display = "block"

      // Calculate Fibonacci retracement levels
      const fibonacciLevels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1];

      // Draw Fibonacci retracement lines
      ctx.beginPath();
      ctx.setLineDash([5, 5]); // Set a dashed line style

      for (const level of fibonacciLevels) {
        const yValue = maxPrice - level * priceRange;
        const y = canvas.height - margin - (yValue - minPrice) * yStep;

        ctx.moveTo(margin, y);
        ctx.lineTo(canvas.width - margin, y);
      
        // Label the line
        ctx.fillStyle = 'black';
        ctx.fillText(`${(level * 100).toFixed(1)}%`, margin - 40, y - 5); // Adjust the x-position
      }

      ctx.stroke();
      ctx.setLineDash([]); // Reset the line style
}

    // Function to calculate linear regression
    function linearRegression(x, y) {
      const n = x.length;
      let sumX = 0;
      let sumY = 0;
      let sumXY = 0;
      let sumX2 = 0;

      for (let i = 0; i < n; i++) {
        sumX += x[i];
        sumY += y[i];
        sumXY += x[i] * y[i];
        sumX2 += x[i] * x[i];
      }

      const slope = (n * sumXY - sumX * sumY) / (n * sumX2 - sumX * sumX);
      const intercept = (sumY - slope * sumX) / n;

      return {
        slope,
        intercept
      };
    }

    // Function to calculate the index based on user input
    function calculateIndex(userInput) {
      // Split the user input into year and month
      const [year, month] = userInput.split('-');
    
      // Calculate the index based on the year and month, assuming January 2000 as the starting point
      const startIndex = (parseInt(year) - 2000) * 12 + (parseInt(month) + 1);
    
      return startIndex;
    }    

    // Function to predict the stock price
    function predictPrice() {
      const prediction = document.getElementById("predictionInput").value;

      // Calculate the x-value based on user input
      let xValue = calculateIndex(prediction)

      // Predict the stock price using linear regression formula
      const predictedPrice = (-regression.slope * xValue + regression.intercept);

      // Display the predicted price
      document.getElementById("prediction").textContent = "$" + predictedPrice.toFixed(2);
    }
  </script>

  <style>
    .prediction-wrapper {
      display: none;
    }
  </style>